# DKTs implementation for FIAT

In [ ]:
from FIAT import polynomial_set, finite_element, functional, dual_set
from FIAT.reference_element import DefaultTriangle
from __future__ import print_function
import numpy as np
np.set_printoptions(precision=2)

In [ ]:
class DKTConstraint(functional.Functional):
    """Functional representing the DKT constraint."""

    def __init__(self, ref_el, x):
        pt_dict = {x: [(1.0, tuple())]}  # {point: [weight, coefficient]}
        super(DKTConstraint, self).__init__(ref_el, tuple(), pt_dict, {}, "DKTConstraint")

    def __call__(self, fn):
        """Evaluate the functional on the function fn."""
        xt, yt = tuple(self.pt_dict.keys())[0]
        r = -fn((xt,yt))
        for x,y in [(0.,0.), (1.,0.), (0.,1.)]:
            dx = functional.PointDerivative(ref_el, (x,y), (1,0))
            dy = functional.PointDerivative(ref_el, (x,y), (0,1))
            r += (fn((x,y)) + dx(fn)*(xt-x) + dy(fn)*(yt-y))/3.

        return r

    # FIXME!!!!
    def to_riesz(self, poly_set):
        """Constructs an array representation of the functional over
        the base of the given polynomial_set so that f(phi) for any
        phi in poly_set is given by a dot product."""
        
        xt, yt = tuple(self.pt_dict.keys())[0]
        f = functional.PointEvaluation(ref_el, (xt,yt))
        r = -f.to_riesz(poly_set)
        print(r)
        for x,y in [(0.,0.), (1.,0.), (0.,1.)]:
            dx = functional.PointDerivative(ref_el, (x,y), (1,0))
            dy = functional.PointDerivative(ref_el, (x,y), (0,1))
            f = functional.PointEvaluation(ref_el, (x,y))
            r += (f.to_riesz(poly_set) + 
                  dx.to_riesz(poly_set)*(xt-x) + 
                  dy.to_riesz(poly_set)*(yt-y))/3.
            print(r)
        return r
   
    def tostr(self):
        x = list(map(str, list(self.pt_dict.keys())[0]))
        return "constraint at (%s)" % (','.join(x),)

In [ ]:
class DKTDualSet(dual_set.DualSet):
    """The dual basis for Discrete Kirchhoff Triangles.
    This class works for simplices of any dimension.
    Nodes are point evaluation and derivatives at vertices,
    plus a constraint at the barycenter.
    Spatial dimension must be 2. """

    def __init__(self, ref_el):
        entity_ids = {}
        nodes = []
        cur = 0  # counter for the nodes (Functionals) added (yes?)

        top = ref_el.get_topology()
        verts = ref_el.get_vertices()
        sd = ref_el.get_spatial_dimension()
        assert sd == 2, "DKT only defined for spatial dimension 2 (was %d)" % sd
        # Vertex dofs: one point evaluation and sd partial derivatives
        entity_ids[0] = {}
        for v in sorted(top[0]):
            nodes.append(functional.PointEvaluation(ref_el, verts[v]))
            pd = functional.PointDerivative
            for i in range(sd):
                alpha = [0] * sd
                alpha[i] = 1
                nodes.append(pd(ref_el, verts[v], alpha))

            entity_ids[0][v] = list(range(cur, cur + 1 + sd))
            cur += sd + 1

        # no edge dofs
        entity_ids[1] = {}
        for e in sorted(top[1]):
            entity_ids[1][e] = []

        # Constraint at the barycenter
        entity_ids[2] = {}
        face = 0
        #pt = ref_el.make_points(2, face, 3)[0]  # Returns (-0.3, -0.3) !?!?
        #nodes.append(DKTConstraint(ref_el, pt))
        nodes.append(DKTConstraint(ref_el, (1./3, 1./3)))
        entity_ids[2] = {face: [cur]}

        super(DKTDualSet, self).__init__(nodes, ref_el, entity_ids)

In [ ]:
class DiscreteKirchhoffTriangle(finite_element.CiarletElement):
    """The Kirchhoff Discrete Triangle element.

    This is only defined over triangles. There are three degrees
    of freedom per vertex: one point evaluation and both partial
    derivatives.
    """

    def __init__(self, ref_el, degree=3):
        # Degree is fixed to 3 for DKTs.
        if ref_el.get_spatial_dimension() != 2:
            raise Exception("DKT requires spatial dimension 2 (was %d)" % sd)
        # TODO: check that we have a triangle!
        poly_set = polynomial_set.ONPolynomialSet(ref_el, 3)
        dual = DKTDualSet(ref_el)

        super(DiscreteKirchhoffTriangle, self).__init__(poly_set, dual, 3)

    def mapping(self):
        """ """
        # FIXME: is this correct?
        mappings = []
        for f in self.dual_basis():
            if isinstance(f, functional.PointEvaluation):
                mappings.append("affine")
            elif isinstance(f, functional.PointDerivative):
                mappings.append("hermite")
            elif isinstance(f, DKTConstraint):
                # HACK: tell FFC to ignore this dof
                mappings.append("zero")
        return mappings

    ###### From here on, things get TRULY hacky
    
    def space_dimension(self):
        """Return the dimension of the finite element space."""
        return 9
    
    def degree(self):
        """Return the degree of the (embedding) polynomial space."""
        return 3

    def dual_basis(self):
        """Return the dual basis (list of functionals) for the finite
        element."""
        return self.dual.get_nodes()[:-1]
    
    def tabulate(self, order, points, entity=None):
        """Return tabulated values of derivatives up to given order of
        basis functions at given points.

        :arg order: The maximum order of derivative.
        :arg points: An iterable of points.
        :arg entity: Optional (dimension, entity number) pair
                     indicating which topological entity of the
                     reference element to tabulate on.  If ``None``,
                     default cell-wise tabulation is performed.
        """
        if entity is None:
            entity = (self.ref_el.get_spatial_dimension(), 0)

        entity_dim, entity_id = entity
        transform = self.ref_el.get_entity_transform(entity_dim, entity_id)
        tabulated = self.poly_set.tabulate(list(map(transform, points)), order)
        # Remove last row (constraint polynomial)
        return {k: v[:-1, :] for k, v in tabulated.items()}

    def get_num_members(self, arg):
        "Return number of members of the expansion set."
        # FIXME: the expansion set has size 10, but we declare a space_dimension of 9
        # This will most likely lead to issues / bugs.
        return self.get_nodal_basis().get_expansion_set().get_num_members(arg)

    def entity_dofs(self):
        """Return the map of topological entities to degrees of
        freedom for the finite element."""
        return {0: {0: [0, 1, 2], 1: [3, 4, 5], 2: [6, 7, 8]},
                1: {0: [], 1: [], 2: []},
                2: {0: []}}

    def entity_closure_dofs(self):
        """Return the map of topological entities to degrees of
        freedom on the closure of those entities for the finite element."""
        return {0: {0: [0, 1, 2], 1: [3, 4, 5], 2: [6, 7, 8]},
                1: {0: [3, 4, 5, 6, 7, 8], 1: [0, 1, 2, 6, 7, 8], 2: [0, 1, 2, 3, 4, 5]},
                2: {0: [0, 1, 2, 3, 4, 5, 6, 7, 8]}}

    def get_formdegree(self):
        """Return the degree of the associated form (FEEC)"""
        return self.formdegree

This won't work:

```python
from FIAT.polynomial_set import PolynomialSet

ref_el = DefaultTriangle()
e = Kirchhoff(ref_el)
ps = e.get_nodal_basis()
coeffs = ps.get_coeffs()[:-1,]
dmats = map(lambda a: a[:-1,], ps.get_dmats())
ps2 = PolynomialSet(ref_el, 
                    ps.get_degree(),
                    ps.get_embedded_degree(),
                    ps.get_expansion_set(),
                    coeffs,
                    dmats)
ps2.tabulate([0.,0.])
-> ValueError: shapes (9,9) and (10,) not aligned: 9 (dim 1) != 10 (dim 0)
```

In [ ]:
ref_el = DefaultTriangle()
fe = DiscreteKirchhoffTriangle(ref_el)
ps = fe.get_nodal_basis()
coeffs = ps.get_coeffs()

In [ ]:
import nbimporter
from shape_functions import compute_trafo, compute_inverse_trafo

def pt(x,y):
    return np.array([x,y], dtype=np.float)

#F = compute_trafo(pt(0,0), pt(1,0), pt(1,1))
Finv = compute_inverse_trafo(pt(0,0), pt(1,0), pt(1,1))

t = np.linspace(0,1,50)
xx, yy = np.meshgrid(t,t)
pts = [[x,y] for y in t for x in t]
zz = fe.tabulate(0, pts)[(0,0)].reshape(9, t.size, t.size)

In [ ]:
import matplotlib.pyplot as pl
%matplotlib inline

def plot_2dshapes(xx, yy, zz, triangle, savefigs=None):
    """ Plots scalar (shape) functions over a triangle.

    Arguments:
    ----------
        savefigs: It not None, a string like "blah-%d.eps"
                  with exactly one format string "%d" in it.
    """
    assert xx.shape == yy.shape, "duh!"
    white = (1.0, 1.0, 1.0, 0.0)
    
    num_funs = zz.shape[0]
    ny, nx = xx.shape
    
    Finv = compute_inverse_trafo(triangle[0], triangle[1], triangle[2])
    def in_triangle(x,y):
        xhat,yhat = Finv(pt(x,y))
        return 0 <= xhat <= 1-yhat and 0 <= yhat
    
    for i, zo in enumerate(zz):
        z = zo.copy()
        fig = pl.figure()
        ax = fig.gca(projection='3d', title="$\phi_{%d}$" % i)
        for m,n in np.ndindex(ny,nx):
            if not in_triangle(xx[m,n], yy[m,n]):
                #print("[%d,%d] --> [%.1f,%.1f] not there." % (m,n,xx[m,n],yy[m,n]))
                z[m,n] = np.nan
        #print(z.round(1))
        vmin, vmax = np.min(z[~np.isnan(z)]), np.max(z[~np.isnan(z)])

        ax.plot_wireframe(xx, yy, z, linewidth=0.2, rstride=2, cstride=2)
        
        # Get rid of the panes
        ax.w_xaxis.set_pane_color(white)
        ax.w_yaxis.set_pane_color(white)
        ax.w_zaxis.set_pane_color(white)

        ax.xaxis.set_ticks([0., 1.])
        ax.yaxis.set_ticks([0., 1.])
        ax.zaxis.set_ticks([vmin, 0, vmax])
        pl.xlabel("x")
        pl.ylabel("y")
        if savefigs is not None:
            pl.savefig(savefigs % i)

In [ ]:
plot_2dshapes(xx,yy,zz, [pt(0,0), pt(1,0), pt(0,1)])

# FFC compilation and dolfin Element

After including the stuff above into FIAT...

In [ ]:
import ufl

import nbimporter
from elements import compile_element
from dolfin import FiniteElement

In [ ]:
de = FiniteElement("DKT", 'triangle', 3)   # Dolfin Element
compile_element(de)

In [ ]:
import FIAT
ref_el = FIAT.reference_element.DefaultTriangle()
vertices = ref_el.get_vertices()
fe = FIAT.DiscreteKirchhoffTriangle(ref_el)
basis_values = fe.tabulate(0, vertices)[(0,) * 2].transpose()
basis_values.round(1)

In [ ]:
from dolfin.compilemodules.jit import jit
from dolfin import UnitSquareMesh
import dolfin.cpp

In [ ]:
ue = ufl.FiniteElement('DKT', 'triangle', 3)
mesh = UnitSquareMesh(1,1)
ufc_element, ufc_dofmap = jit(ue, mpi_comm=mesh.mpi_comm())
dolfin_element = dolfin.cpp.FiniteElement(ufc_element)

In [ ]:
ddm = dolfin.cpp.DofMap(ufc_dofmap, mesh)

In [ ]:
ddm.cell_dofs(0), ddm.cell_dofs(1)

# FAIL:

In [ ]:
from dolfin import *
import nbimporter
from boundary import apply_dirichlet_hermite
from utils import * 

V = FunctionSpace(UnitSquareMesh(20, 20), 'DKT', 3)

top = lambda x: near(x[1], 1.)
right = lambda x: near(x[0], 1.)
bottom = lambda x: near(x[1], 0.)
left = lambda x: near(x[0], 0.)

u0 = project(Constant(2.0), V)
bc = DirichletBC(V, u0, fnor(top, right, bottom, left))

class td_subdomain(SubDomain):
    """ Top and bottom sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[1], 0.) or near(x[1], 1.)

class lr_subdomain(SubDomain):
    """ Left and right sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[0], 0.) or near(x[0], 1.)

# We will use this to mark boundaries for natural boundary conditions
exterior_facet_domains = FacetFunction("uint", V.mesh(), value=1)
#exterior_facet_domains.set_all(1)

natural_boundary = td_subdomain()
natural_boundary.mark(exterior_facet_domains, 0)
ds_neu = ds(subdomain_data = exterior_facet_domains, subdomain_id=0)

u = TrialFunction(V)
v = TestFunction(V)
#g = make_constant(0.0, V)
f = project(Constant(4.0), V)
a = inner(grad(u), grad(v))*dx
F = f*v*dx # + g*v*ds_neu

# Compute solution
A = assemble(a)
b = assemble(F)
Ac = A.copy()
print("Applying BCs... ", end='')
apply_dirichlet_hermite(A, b, bc)
print("done.")
u = Function(V)
uv = u.vector()
_ = solve(A, uv, b)

In [ ]:
%matplotlib inline
plot(u, cmap='bone')

In [ ]:
import matplotlib.pyplot as pl
import numpy as np

# Double fail

In [ ]:
xx = np.linspace(0,1,100)
pl.plot(xx, [u(x,0.5) for x in xx])

In [ ]:
g = project(Constant(1.0), V)

In [ ]:
assemble(g*dx)

In [ ]:
plot(g)

In [ ]:
gv = g.vector()
gva = gv.array().copy()
gva[:] = 0.
gva[::3] = 1.
gv.set_local(gva)
gv.array().round(1)

# Back to square 1 (by 1)

In [ ]:
V = FunctionSpace(UnitSquareMesh(1,1), 'DKT', 3)

In [ ]:
u = Function(V)
uv = u.vector()
vals = np.zeros(V.dim())
dm = V.dofmap()
c1, c2 = dm.cell_dofs(0), dm.cell_dofs(1)
vals[c1[::3]] = 1.
vals[c2[::3]] = 1.
uv.set_local(vals)

In [ ]:
xx = np.linspace(0,1,100)
for y in np.linspace(0,1,4):
    pl.plot(xx, [u(x, y) for x in xx],label='$y=%.1f$'%y)
_ = pl.legend()

In [ ]:
import nbimporter
from boundary import plot_dofs

In [ ]:
plot_dofs(V, [9,10,11])